# Описание 

Попробуем решить несколько типичных задач поиска домов, которые нам предстоят.

Поймём, нам может пригодиться API Яндекс.Карт.

# Импорты

In [239]:
import os
import requests
import re
from getpass import getpass

from num2words import num2words

import pandas as pd

'одиннадцатый'

# Данные

In [6]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "data")
sep = "\t"
house_tags_path = os.path.join(data_path, "77_ht.tsv")
house_addresses_path = os.path.join(data_path, "77_ha.tsv")

In [7]:
house_tags_ds = pd.read_csv(house_tags_path, sep=sep)
house_addresses_ds = pd.read_csv(house_addresses_path, sep=sep)

# Подготовка

Запишем алфавит -- он нам понадобится для поиска номера буквы.

In [135]:
alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
alphabet_dict = {letter: index + 1
    for index, letter in enumerate(alphabet)
}

Преобразование порядковых числительных

In [108]:
def num2text(input_string):
    sex = 0
    if input_string[-1] == "я":
        sex = 1
        
    text = num2words(int(input_string.split("-")[0]), lang='ru', to="ordinal")
    
    if sex == 0:
        return text
    if (text[-2:] == "ый") or (text[-2:] == "ой"):
        return text[:-2] + "ая"
    return text[:-2] + "ья"

Преобразуем названия улиц и выделим номера домов.

In [151]:
alnum_pattern = re.compile(r'[\W_]+')
ordinal_pattern = re.compile(r'[0-9]+-[\w]')
num_pattern = re.compile(r'[0-9]+')

def is_strictly_alnum(input_string):
    stripped_string = pattern.sub('', input_string)
    return not(stripped_string.isdigit() or stripped_string.isalpha())

house_addresses_ds["correct"] = house_addresses_ds.apply(
    lambda x: not x["house_num"].isalpha() and (len(num_pattern.findall(x["street_name"])) == 0),
    axis=1,
)

house_addresses_ds["ordinal_number"] = house_addresses_ds["street_name"].apply(
    lambda x: len(ordinal_pattern.findall(x)) > 0
)

In [152]:
house_addresses_ds.groupby(["correct", "ordinal_number", "is_active"], as_index=False)["oid"].count()

,correct,ordinal_number,is_active,oid
0,False,False,False,31280
1,False,False,True,48568
2,False,True,False,6236
3,False,True,True,21570
4,True,False,False,51146
5,True,False,True,187557


In [154]:
house_addresses_ds[
    ~house_addresses_ds["correct"] 
    & house_addresses_ds["ordinal_number"]
    & house_addresses_ds["is_active"]
].sample(5)

,oid,street_name,street_type,house_num,house_type,house_add_1_num_,house_add_1_type,house_add_2_num_,house_add_2_type,is_active,street_name_processed,correct,ordinal_number
158377,38351015,2-й Казачий,пер,3,д.,2,стр.,NaN,NaN,True,2-й казачий,False,True
124613,36550718,1-я Боевская,ул,1,д.,3,стр.,NaN,NaN,True,1-я боевская,False,True
212423,46638716,2-я Тверская-Ямская,ул,46,д.,1,стр.,NaN,NaN,True,2-я тверская-ямская,False,True
101206,41638194,2-я Магистральная,ул,5А,д.,4,стр.,NaN,NaN,True,2-я магистральная,False,True
149236,164945197,2-я Фиалковая,ул,42,д.,NaN,NaN,NaN,NaN,True,2-я фиалковая,False,True


In [155]:
house_addresses_ds[
    ~house_addresses_ds["correct"] 
    & ~house_addresses_ds["ordinal_number"]
    & house_addresses_ds["is_active"]
].sample(5)

,oid,street_name,street_type,house_num,house_type,house_add_1_num_,house_add_1_type,house_add_2_num_,house_add_2_type,is_active,street_name_processed,correct,ordinal_number
39464,97636614,81,кв-л,106А,д.,1,стр.,NaN,NaN,True,81,False,False
346324,53292103,22,кв-л,48,д.,1,стр.,NaN,NaN,True,22,False,False
15519,52743200,72,кв-л,121,д.,1,стр.,NaN,NaN,True,72,False,False
241849,30354565,9,кв-л,125,д.,1,стр.,NaN,NaN,True,9,False,False
318480,103503264,119,кв-л,25А,д.,2,стр.,NaN,NaN,True,119,False,False


Так как сложно определить порядок, куда нужно поставить тип улицы в такое название, для первого подхода попробуем игнорировать их вообще.

Приведём названия и номера домов к подходящему виду.

In [156]:
house_addresses_filtered_ds = house_addresses_ds[
    house_addresses_ds["correct"] 
    & ~house_addresses_ds["ordinal_number"]
].copy()

In [157]:
house_addresses_filtered_ds["street_name_processed"] = house_addresses_filtered_ds["street_name"].apply(lambda x: x.lower().replace(" ", ""))
house_addresses_filtered_ds["house_num_processed"] = house_addresses_filtered_ds["house_num"].apply(lambda x: int(re.findall("[0-9]+", x)[0]))

# Анализ

Попробуем решить несколько задач.

## Задача первая

Воспользуемся заданиями из https://sledopyt-moscow.ru/competitions/96/tasks_96.html

В районах Раменки и Дорогомилово определить возможные дома:
* X3+X5+1 (Студенческая улица);
* X4+3 (Можайский Вал);
* X6+4 (улица Братьев Фонченко);
* X5-X2-4 (Большая Дорогомиловская улица);
* X3:2 (Можайский Вал).

Здесь XK -- это порядковый номер в алфавите K-ой буквы в названии улицы.

Отфильтруем дома:

In [158]:
houses_in_area_ds = house_addresses_filtered_ds.merge(
        house_tags_ds[house_tags_ds["tag"].isin(["Раменки", "Дорогомилово"])][["oid"]]
    )
print(len(houses_in_area_ds))

2708


In [189]:
def check_equation(street_name, house_num, equation_pattern, indices):
    if max(indices) > len(street_name):
        return False
    street_nums = [alphabet_dict[street_name[i-1]] for i in indices]
    equation = equation_pattern.format(*street_nums)
    return eval(equation) == house_num

In [169]:
def get_address(x):
    parts = []
    parts.append("{} {}".format(x["street_name"], x["street_type"]))
    parts.append("{} {}".format(x["house_num"], x["house_type"]))
    if not pd.isnull(x["house_add_1_num_"]):
        parts.append("{} {}".format(x["house_add_1_num_"], x["house_add_1_type"]))
    if not pd.isnull(x["house_add_2_num_"]):
        parts.append("{} {}".format(x["house_add_2_num_"], x["house_add_2_type"]))
    return ",".join(parts)

### X3+X5+1

In [206]:
equation_pattern = "{}+{}+1"
letter_indices = [3, 5]

Студенческая улица

In [207]:
houses_in_area_ds[
    houses_in_area_ds.apply(
        lambda x: check_equation(x["street_name_processed"], x["house_num_processed"], equation_pattern, letter_indices),
        axis=1,
    )
    & houses_in_area_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates()

,street_name,house_num_processed
1065,Мосфильмовская,30
1215,Кутузовский,30
1422,Студенческая,28
1500,Мичуринский,44
2230,Малая Дорогомиловская,47
2504,Довженко,10


### X4+3

In [208]:
equation_pattern = "{}+3"
letter_indices = [4]

Можайский Вал

In [209]:
houses_in_area_ds[
    houses_in_area_ds.apply(
        lambda x: check_equation(x["street_name_processed"], x["house_num_processed"], equation_pattern, letter_indices),
        axis=1,
    )
    & houses_in_area_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates()

,street_name,house_num_processed
1,Неверовского,9
171,Раменки,9
175,Косыгина,32
350,Кутузовский,24
564,Мосфильмовская,25
927,Можайский Вал,4
1132,Поклонная,16
1302,Можайский,4
2185,Дунаевского,4
2356,Мичуринский,24


### X6+4

In [210]:
equation_pattern = "{}+4"
letter_indices = [6]

улица Братьев Фонченко

In [211]:
houses_in_area_ds[
    houses_in_area_ds.apply(
        lambda x: check_equation(x["street_name_processed"], x["house_num_processed"], equation_pattern, letter_indices),
        axis=1,
    )
    & houses_in_area_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates()

,street_name,house_num_processed
79,Бережковская,16
164,Мосфильмовская,17
332,Студенческая,19
374,Большая Дорогомиловская,5
389,Пырьева,7
441,Братьев Фонченко,10
616,Кутузовский,20
743,Дунаевского,7
909,Ломоносовский,20
1107,Киевская,16


### X5-X2-4

In [212]:
equation_pattern = "{}-{}-4"
letter_indices = [5, 2]

Большая Дорогомиловская улица

In [213]:
houses_in_area_ds[
    houses_in_area_ds.apply(
        lambda x: check_equation(x["street_name_processed"], x["house_num_processed"], equation_pattern, letter_indices),
        axis=1,
    )
    & houses_in_area_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates()

,street_name,house_num_processed
23,Киевская,5
585,Братьев Фонченко,8
652,Неверовского,8
1045,Мичуринский,4
1268,Большая Дорогомиловская,6
1379,Генерала Дорохова,8
2029,Евразии,2
2076,Резервный,8


### X3:2

In [214]:
equation_pattern = "{}/2"
letter_indices = [3]

Можайский Вал

In [215]:
houses_in_area_ds[
    houses_in_area_ds.apply(
        lambda x: check_equation(x["street_name_processed"], x["house_num_processed"], equation_pattern, letter_indices),
        axis=1,
    )
    & houses_in_area_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates()

,street_name,house_num_processed
61,Тараса Шевченко,9
96,Раменки,7
192,Пырьева,9
336,Победы,1
539,Кутузовский,10
574,Столетова,8
927,Можайский Вал,4
1272,Раевского,3
1302,Можайский,4
2198,Поклонная,6


### Итоги

По первому типу задач мы в целом нашли всякий раз правильные улицы, фильтрация нам не помешала. Впрочем, подходящих улиц всякий раз довольно много.

## Задача вторая

Мы хотим найти улицы, на которых есть нужный дом. При этом результаты хорошо бы ранжировать в порядке близости к заданной геолокации.

Задания:
* Дом 38 \[по улице Фридриха Энгельса\], точка пользователя: 55.764956, 37.719036, [ссылка](https://www.runcity.org/ru/events/msk2023/routes/cp542/);
* Дом 31 \[по Лениногорской улице\], точка пользователя: 55.705709, 37.928431, [ссылка](https://www.runcity.org/ru/events/msk2023/routes/cp514/).

Для удобства ограничимся районами игры -- ЮВАО, ЦАО, ВАО.

In [270]:
areas_UVAO = [
	"Выхино-Жулебино",
	"Капотня",
	"Кузьминки",
	"Лефортово",
	"Люблино",
	"Марьино",
	"Некрасовка",
	"Нижегородский",
	"Печатники",
	"Рязанский",
	"Текстильщики",
	"Южнопортовый",
]
areas_VAO = [
    "Богородское",
	"Вешняки",
	"Восточное Измайлово",
	"Восточный",
	"Гольяново",
	"Ивановское",
	"Измайлово",
	"Косино-Ухтомский",
	"Метрогородок",
	"Новогиреево",
	"Новокосино",
	"Перово",
	"Преображенское",
	"Северное Измайлово",
	"Соколиная гора",
	"Сокольники",
]
areas_CAO = [
    "Арбат",
	"Басманный",
	"Замоскворечье",
	"Красносельский",
	"Мещанский",
	"Пресненский",
	"Таганский",
	"Тверской",
	"Хамовники",
	"Якиманка",
]

In [271]:
houses_in_UVAO_ds = house_addresses_filtered_ds.merge(
        house_tags_ds[house_tags_ds["tag"].isin(areas_UVAO)][["oid"]]
    )
print(len(houses_in_UVAO_ds))

houses_in_VAO_ds = house_addresses_filtered_ds.merge(
        house_tags_ds[house_tags_ds["tag"].isin(areas_VAO)][["oid"]]
    )
print(len(houses_in_VAO_ds))

houses_in_CAO_ds = house_addresses_filtered_ds.merge(
        house_tags_ds[house_tags_ds["tag"].isin(areas_CAO)][["oid"]]
    )
print(len(houses_in_CAO_ds))

13801
17110
22048


### Подготовка ранжирования

In [277]:
houses_in_UVAO_ds[
    (houses_in_UVAO_ds["house_num_processed"] == 31)
    & houses_in_UVAO_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates().shape

(40, 2)

In [276]:
houses_in_VAO_ds[
    (houses_in_VAO_ds["house_num_processed"] == 31)
    & houses_in_VAO_ds["is_active"]
][["street_name", "house_num_processed"]].drop_duplicates().shape

(46, 2)

In [240]:
GEOCODER_API = getpass()

 ········


In [287]:
geocoder_url = "https://geocode-maps.yandex.ru/v1"

In [309]:
def get_coordinates(street_name, street_type, house_num):
    params = {
        "apikey": GEOCODER_API,
        "geocode": f"Москва, {street_name} {street_type}, {house_num}",
        "lang": "ru_RU",
        "format": "json",
        "results": 1,
    }
    response = requests.get(geocoder_url, params=params)
    return tuple(
        float(x) for x in
        response.json()["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split()
    )[::-1]

def count_dist(a_coords, b_coords):
    return ((a_coords[0] - b_coords[0]) ** 2 + (a_coords[1] - b_coords[1]) ** 2) ** 0.5 * 111

### Дом 31

In [301]:
house_num = 31
start_coords = (55.705709, 37.928431)

In [297]:
candidates_ds = houses_in_UVAO_ds[
    (houses_in_UVAO_ds["house_num_processed"] == house_num)
    & houses_in_UVAO_ds["is_active"]
][["street_name", "street_type"]].drop_duplicates()

In [298]:
candidates_ds["coords"] = candidates_ds.apply(
    lambda x: get_coordinates(x["street_name"], x["street_type"], house_num),
    axis=1,
)

In [310]:
candidates_ds["dist"] = candidates_ds["coords"].apply(lambda x: count_dist(x, start_coords))

In [311]:
candidates_ds.sort_values(by="dist")

,street_name,street_type,coords,dist
5683,Рождественская,ул,"(55.705919, 37.931731)",0.367041
1983,Покровская,ул,"(55.705001, 37.922335)",0.681204
2991,Жулебинский,б-р,"(55.686376, 37.848044)",9.177381
838,Ферганская,ул,"(55.701739, 37.829781)",10.959013
754,Головачёва,ул,"(55.675758, 37.816648)",12.845583
13002,Ташкентская,ул,"(55.695844, 37.804934)",13.751833
4121,Михайлова,ул,"(55.725995, 37.774203)",17.266762
12506,Ставропольская,ул,"(55.682331, 37.773574)",17.383898
5203,Зеленодольская,ул,"(55.707314, 37.770007)",17.585966
11464,Зарайская,ул,"(55.730765, 37.771032)",17.691272


In [312]:
candidates_ds = houses_in_VAO_ds[
    (houses_in_VAO_ds["house_num_processed"] == house_num)
    & houses_in_VAO_ds["is_active"]
][["street_name", "street_type"]].drop_duplicates()

In [313]:
candidates_ds["coords"] = candidates_ds.apply(
    lambda x: get_coordinates(x["street_name"], x["street_type"], house_num),
    axis=1,
)

In [314]:
candidates_ds["dist"] = candidates_ds["coords"].apply(lambda x: count_dist(x, start_coords))

In [315]:
candidates_ds.sort_values(by="dist")

,street_name,street_type,coords,dist
439,Красковская,ул,"(55.71306, 37.900434)",3.213003
14417,Лухмановская,ул,"(55.724885, 37.90312)",3.524780
1164,Третьего Интернационала,ул,"(55.704022, 37.882872)",5.060515
4956,Каскадная,ул,"(55.70598, 37.881659)",5.191779
8024,Лыткаринская,ул,"(55.701998, 37.879889)",5.403885
7201,Камова,ул,"(55.702125, 37.875874)",5.847376
4728,Розы Люксембург,ул,"(55.700628, 37.871005)",6.399188
15948,Златоустовская,ул,"(55.699132, 37.870879)",6.429851
10997,Чебоксарская,ул,"(55.702916, 37.869882)",6.506329
10472,Салтыковская,ул,"(55.73995, 37.874544)",7.086857


Итак, улица нашлась, но не слишком близко -- а много было вариантов и ближе.

### Дом 38

In [317]:
house_num = 38
start_coords = (55.764956, 37.719036)

In [318]:
candidates_ds = houses_in_VAO_ds[
    (houses_in_VAO_ds["house_num_processed"] == house_num)
    & houses_in_VAO_ds["is_active"]
][["street_name", "street_type"]].drop_duplicates()

In [319]:
candidates_ds["coords"] = candidates_ds.apply(
    lambda x: get_coordinates(x["street_name"], x["street_type"], house_num),
    axis=1,
)

In [320]:
candidates_ds["dist"] = candidates_ds["coords"].apply(lambda x: count_dist(x, start_coords))

In [321]:
candidates_ds.sort_values(by="dist")

,street_name,street_type,coords,dist
2543,Большая Семёновская,ул,"(55.781291, 37.711518)",1.996003
2144,Щербаковская,ул,"(55.780684, 37.732386)",2.289918
41,Энтузиастов,ш,"(55.755419, 37.738989)",2.454773
1851,Уткина,ул,"(55.764637, 37.744145)",2.787324
1939,Ткацкая,ул,"(55.786607, 37.737659)",3.169982
11631,Мироновская,ул,"(55.790924, 37.733042)",3.274980
5740,Хромова,ул,"(55.799988, 37.723726)",3.923245
804,Знаменская,ул,"(55.801981, 37.722496)",4.127681
25,Краснобогатырская,ул,"(55.81037, 37.702544)",5.363055
14976,Маршала Рокоссовского,б-р,"(55.818483, 37.714114)",5.966563


In [323]:
candidates_ds = houses_in_CAO_ds[
    (houses_in_CAO_ds["house_num_processed"] == house_num)
    & houses_in_CAO_ds["is_active"]
][["street_name", "street_type"]].drop_duplicates()

In [324]:
candidates_ds["coords"] = candidates_ds.apply(
    lambda x: get_coordinates(x["street_name"], x["street_type"], house_num),
    axis=1,
)

In [325]:
candidates_ds["dist"] = candidates_ds["coords"].apply(lambda x: count_dist(x, start_coords))

In [326]:
candidates_ds.sort_values(by="dist")

,street_name,street_type,coords,dist
4033,Большая Почтовая,ул,"(55.781479, 37.696885)",3.067451
7524,Фридриха Энгельса,ул,"(55.775165, 37.688045)",3.621843
306,Бакунинская,ул,"(55.775464, 37.68677)",3.766668
3127,Новорогожская,ул,"(55.738753, 37.691899)",4.187237
12853,Международная,ул,"(55.741657, 37.688494)",4.263985
6170,Рабочая,ул,"(55.741161, 37.685916)",4.526754
5339,Бауманская,ул,"(55.771626, 37.678523)",4.557482
484,Золоторожский Вал,ул,"(55.747915, 37.679808)",4.747416
10334,Большая Калитниковская,ул,"(55.736169, 37.686195)",4.847565
12668,Новорязанская,ул,"(55.771175, 37.670088)",5.476905


Мда, тут она поближе -- но всё равно есть с пяток вариантов и более близких. 

### Заключение

В текущем формате данные мало полезны -- и требуют довольно много лимитированных запросов в геокодер.

Возможно, эту задачу можно было бы улучшить за счёт сильного ограничения области поиска. Но в целом кажется не очень перспективным.